Minimal TTGO or Heltec send any packet and display it on the OLED

Uses the jupyter_micropython_kernel to hack the code in
https://github.com/goatchurchprime/jupyter_micropython_kernel/

Depends on MicroPython for ESP32, 
and https://github.com/goatchurchprime/LightLora_MicroPython
and ssd1306.py OLED library (included)

Thes packets are picked up by 
https://github.com/kersing/ESP-1ch-Gateway-v5.0/tree/master/ESP-sc-gway (although from not farther than one room away)

Useful set of documentes: https://github.com/Wei1234c/MQTT_bridged_LoRa_networks_with_MicroPython_on_ESP32/tree/master/references

In [2]:
# Run this to upload the files to the sending mode 
# (after you have run the esp32_commissioning notebook)

%serialconnect
%sendtofile --quiet --source ssd1306.py
%sendtofile --mkdir --quiet --source /home/julian/extrepositories/LightLora_MicroPython/LightLora/ lib/LightLora/
%sendtofile --source utils.py
%sendtofile --source mainsend.py main.py




***Connection broken [Input/output error]
You may need to reconnectserial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.
Sent 168 lines (5477 bytes) to ssd1306.py.
Sent 106 lines (2864 bytes) to lib/LightLora/lorautil.py.
Sent 397 lines (13311 bytes) to lib/LightLora/sx127x.py.
Sent 64 lines (2087 bytes) to lib/LightLora/spicontrol.py.
Sent 0 lines (0 bytes) to lib/LightLora/__init__.py.


In [25]:
%sendtofile --source mainsend.py main.py


Sent 29 lines (712 bytes) to main.py.


### Everything below is for hacking the code 
that we are putting into main.py (which is 
the script that runs on startup)

We are trying to correlate the code between the things network code
https://github.com/kersing/ESP-1ch-Gateway-v5.0/tree/master/ESP-sc-gway

and the code in LightLora_MicroPython to do with setting the 
values in the Lora chip.  

At the moment the ESP-gateway sees the incoming packets, 
but the range is no more than across a single room.  
(However, the ESP-gateway is able to collect unknown packets 
from distant places, so we think it's got more appropriate 
settings)

In [27]:
%serialconnect
# start here for interaction


Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [24]:
import utils
params = {  'frequency': 868100000, 
            'tx_power_level': 14, 
            'signal_bandwidth': 125E3, 
            'spreading_factor': 7, 
            'coding_rate': 5, 
            'preamble_length': 8,
            'implicitHeader': False, 
            'sync_word': 0x34,    # from _loraModem
            'enable_CRC': True }
utils.MakeLora(params)

Writing mainsend.py



In [20]:
utils.lora.init()


In [21]:
utils.lora.onReceive(utils._doReceive)
utils.lora.onTransmit(utils._doTransmit)
utils.lora.receive()


In [23]:
n = 1999
txt = "Hello %d there"%n
btxt = txt.encode()
btxt = b"\x90\x91\x92"
utils.sendPacket(0xfe, 0x41, btxt) 


In [19]:
%sendtofile --append main.py

from lib.LightLora import lorautil, spicontrol
lru = lorautil.LoraUtil()

Sent 2 lines (72 bytes) to main.py.


In [54]:
lru.lora.setSyncWord(0x36)

In [53]:
n = 1999
txt = "Hello %d there"%n
btxt = txt.encode()
btxt = b"\x90\x91\x92"
lru.sendPacket(0xfe, 0x41, btxt) 


Sent 29 lines (712 bytes) to main.py.


In [31]:
utils.writeoled("kk")

In [35]:
s = utils.sx127x
l = utils.lora

In [42]:
#ASSERT((readRegister(REG_OPMODE) & OPMODE_LORA) != 0);
print(l.readRegister(s.REG_OP_MODE)&0x80)
# opmode(OPMODE_STANDBY);
#s.OPMODE_STANDBY
setRate(sfTx=7, crc=True) # spreading factor
REG_MODEM_CONFIG_1 = 0x1d
REG_MODEM_CONFIG_2 = 0x1e
REG_RSSI_WIDEBAND = 0x2c
REG_DETECTION_OPTIMIZE = 0x31
REG_DETECTION_THRESHOLD = 0x37
def setSpreadingFactor(self, sf):
    sf = min(max(sf, 6), 12)
    self.writeRegister(REG_DETECTION_OPTIMIZE, 0xc5 if sf == 6 else 0xc3)
    self.writeRegister(REG_DETECTION_THRESHOLD, 0x0c if sf == 6 else 0x0a)
    self.writeRegister(REG_MODEM_CONFIG_2, (self.readRegister(REG_MODEM_CONFIG_2) & 0x0f) | ((sf << 4) & 0xf0))
#define REG_MODEM_CONFIG1           0x1D
#define REG_MODEM_CONFIG2           0x1E

    
		mc1= 0x0A;				// SX1276_MC1_BW_250 0x80 | SX1276_MC1_CR_4_5 0x02
		mc2= ((sf<<4) | crc) % 0xFF;
		// SX1276_MC1_BW_250 0x80 | SX1276_MC1_CR_4_5 0x02 | SX1276_MC1_IMPLICIT_HEADER_MODE_ON 0x01
        if (sf == SF11 || sf == SF12) { mc1= 0x0B; }			        
	writeRegister(REG_MODEM_CONFIG1, (uint8_t) mc1);
	writeRegister(REG_MODEM_CONFIG2, (uint8_t) mc2);
	writeRegister(REG_MODEM_CONFIG3, (uint8_t) mc3);
    

128


Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
AttributeError: 'module' object has no attribute 'OPMODE_STANDBY'


In [ ]:
# Pow setting looks like it does same thing

REG_PA_CONFIG = 0x09
#define REG_PAC                     0x09

void setPow(uint8_t powe)
{
	if (powe >= 16) powe = 15;
	//if (powe >= 15) powe = 14;
	else if (powe < 2) powe =2;
	
	ASSERT((powe>=2)&&(powe<=15));
	
	uint8_t pac = (0x80 | (powe & 0xF)) & 0xFF;
	writeRegister(REG_PAC, (uint8_t)pac);								// set 0x09 to pac
			self.writeRegister(REG_PA_CONFIG, PA_BOOST | (level - 2))

    
	def setTxPower(self, level, outputPin=PA_OUTPUT_PA_BOOST_PIN):
		if outputPin == PA_OUTPUT_RFO_PIN:
			# RFO
			level = min(max(level, 0), 14)
			self.writeRegister(REG_PA_CONFIG, 0x70 | level)
		else:
			# PA BOOST
			level = min(max(level, 2), 17)
			self.writeRegister(REG_PA_CONFIG, PA_BOOST | (level - 2))
    

In [ ]:
# code from the ESP-1ch gateway
// 2. enter standby mode (required for FIFO loading))
	opmode(OPMODE_STANDBY);									// set 0x01 to 0x01
	
	// 3. Init spreading factor and other Modem setting
	setRate(sfTx=7, crc=True);
	
	// Frquency hopping
	//writeRegister(REG_HOP_PERIOD, (uint8_t) 0x00);		// set 0x24 to 0x00 only for receivers
	
	// 4. Init Frequency, config channel
	setFreq(freq);

	// 6. Set power level, REG_PAC
	setPow(powe);
	
	// 7. prevent node to node communication
	writeRegister(REG_INVERTIQ, (uint8_t) iiq);						// 0x33, (0x27 or 0x40)
	
	// 8. set the IRQ mapping DIO0=TxDone DIO1=NOP DIO2=NOP (or lesss for 1ch gateway)
    writeRegister(REG_DIO_MAPPING_1, (uint8_t)(MAP_DIO0_LORA_TXDONE|MAP_DIO1_LORA_NOP|MAP_DIO2_LORA_NOP));
	
	// 9. clear all radio IRQ flags
    writeRegister(REG_IRQ_FLAGS, (uint8_t) 0xFF);
	
	// 10. mask all IRQs but TxDone
    writeRegister(REG_IRQ_FLAGS_MASK, (uint8_t) ~IRQ_LORA_TXDONE_MASK);
	
	// txLora
	opmode(OPMODE_FSTX);									// set 0x01 to 0x02 (actual value becomes 0x82)
	
	// 11, 12, 13, 14. write the buffer to the FiFo
	sendPkt(payLoad, payLength);

	// 15. wait extra delay out. The delayMicroseconds timer is accurate until 16383 uSec.
	loraWait(tmst);
	
	//Set the base addres of the transmit buffer in FIFO
    writeRegister(REG_FIFO_ADDR_PTR, (uint8_t) readRegister(REG_FIFO_TX_BASE_AD));	// set 0x0D to 0x0F (contains 0x80);	
	
	//For TX we have to set the PAYLOAD_LENGTH
    writeRegister(REG_PAYLOAD_LENGTH, (uint8_t) payLength);		// set 0x22, max 0x40==64Byte long
	
	//For TX we have to set the MAX_PAYLOAD_LENGTH
    writeRegister(REG_MAX_PAYLOAD_LENGTH, (uint8_t) MAX_PAYLOAD_LENGTH);	// set 0x22, max 0x40==64Byte long
	
	// Reset the IRQ register
	//writeRegister(REG_IRQ_FLAGS, 0xFF);						// set 0x12 to 0xFF
	writeRegister(REG_IRQ_FLAGS_MASK, (uint8_t) 0x00);			// Clear the mask
	writeRegister(REG_IRQ_FLAGS, (uint8_t) IRQ_LORA_TXDONE_MASK);// set 0x12 to 0x08
	
	// 16. Initiate actual transmission of FiFo
	opmode(OPMODE_TX);											// set 0x01 to 0x03 (actual value becomes 0x83)

		txLoraModem(
			LoraDown.payLoad,
			LoraDown.payLength,
			LoraDown.tmst,
			LoraDown.sfTx,
			LoraDown.powe,
			LoraDown.fff,
			LoraDown.crc,
			LoraDown.iiq
		);


In [ ]:
		# config
		self.setFrequency(self._useParam('frequency'))
		self.setSignalBandwidth(self._useParam('signal_bandwidth'))

		# set LNA boost
		self.writeRegister(REG_LNA, self.readRegister(REG_LNA) | 0x03)

		# set auto AGC
		self.writeRegister(REG_MODEM_CONFIG_3, 0x04)

		self.setTxPower(self._useParam('tx_power_level'))
		self._implicitHeaderMode = None
		self.implicitHeaderMode(self._useParam('implicitHeader'))
		self.setSpreadingFactor(self._useParam('spreading_factor'))
		self.setCodingRate(self._useParam('coding_rate'))
		self.setPreambleLength(self._useParam('preamble_length'))
		self.setSyncWord(self._useParam('sync_word'))
		self.enableCRC(self._useParam('enable_CRC'))

		# set base addresses
		self.writeRegister(REG_FIFO_TX_BASE_ADDR, FifoTxBaseAddr)
		self.writeRegister(REG_FIFO_RX_BASE_ADDR, FifoRxBaseAddr)

		self.standby()
